In [1]:
!pip install openai pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00


In [2]:
# 设置OpenAI KEY环境变量
import os
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


# 本示例，我们将展示如何使用OpenAI API进行微调OpenAI的模型：

PS：

1）微调的数据集合微调后的模型必须保存到OpenAI的服务器上，像调用OpenAI模型一样调用微调后的模型，因此还是需要付费的；

2）只能微调ada、babbage、curie和davinci基础模型（未经过RLHF），不能微调InstructGPT系列



In [ ]:
from openai import OpenAI

client = OpenAI()

def chat_completion(prompt, model="gpt-4", temperature=0):
    res = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
    )
    print(res.choices[0].message.content)
import pandas as pd

本案例我们将为一家电子邮件营销机构制作一个文本生成工具。

我们使用GPT-3.5 Turbo模型来生成数据（**工作中应该有专家进行标注**），并使用这些数据微调模型

In [ ]:
# 首先定义3个列表，分别对应商店类型、商店所在城市和商店规模
l_sector = ['Grocery Stores', 'Restaurants', 'Fast Food Restaurants', 'Pharmacies', 'Service Stations (Fuel)', 'Electronics Stores']
l_city = ['Brussels', 'Paris', 'Berlin']
l_size = ['small', 'medium', 'large']

# 定义生成合成数据的Prompt
f_prompt = """
Role: You are an expert content writer with extensive direct marketing experience. You have strong writing skills, creativity, adaptability to different tones and styles, and a deep understanding of audience needs and preferences for effective direct campaigns.
Context: You have to write a short message in maximum 2 sentences for a direct marketing campaign to sell a new e-commerce payment service to stores.
The target stores have the three following characteristics:
- The sector of activity: {sector}
- The city where the stores are located: {city}
- The size of the stores: {size}
Task: Writes the short message for the direct marketing campaign. To write this message, use your skills defined in your role! It is very important that the messages you produce take into account the product you want to sell and the characteristics of the store you want to write to.
"""

# 构造微调数据集
f_sub_prompt = "{sector}, {city}, {size}"

#
df = pd.DataFrame()
for sector in l_sector:
 for city in l_city:
  for size in l_size:
   for i in range(3): ## 3 times each
    prompt = f_prompt.format(sector=sector, city=city, size=size)
    sub_prompt = f_sub_prompt.format(sector=sector, city=city, size=size)

    response_txt = chat_completion(prompt, model='gpt-3.5-turbo', temperature=1)

    # sub_prompt和response_txt添加到out_openai_completion.csv文件中，作为微调数据集
    new_row = {
      'prompt':sub_prompt,
      'completion':response_txt}
    new_row = pd.DataFrame([new_row])
    df = pd.concat([df, new_row], axis=0, ignore_index=True)

df.to_csv("out_openai_completion.csv")


# 接下来，上传数据到OpenAI服务器

微调数据需要是JSONL格式，OpenAI提供了一个工具，可以把各种文件格式（CSV、TSV、XLSX、JSON或者JSONL）作为输入，**只要他们包含提示词和文本补全列**

## CSV格式转换为JSONL

In [4]:
!openai tools fine_tunes.prepare_data -f out_openai_completion.csv
# 微调结束后会生成out_openai_completion_prepared.jsonl文件

## 上传微调数据集并进行微调

In [ ]:
ft_file = openai.File.create(file=open("out_openai_completion_prepared.jsonl", "rb"), purpose="fine-tune") # 微调时，purpose设置为fine-tune。此函数会输出一个字典，可以通过id检索文件
# 列出已上传的文件
!openai.File.list()
openai.FineTune.create(training_file=ft_file["id"], model="davinci", suffix="direct_marketing") # training_file必须格式化为JSONL文件；model是用于微调的模型；suffix最多由40个字符组成的字符串，它将被添加到微调后模型名称中。

In [ ]:
!openai api fine_tunes.create -t out_openai_completion_prepared.jsonl -m davinci --suffix "direct_marketing"
# 此时使用Ctrl+C会断开与OpenAI服务器的连接，但不会中断微调过程。要获取正在运行的微调作业，可以使用如下命令：
!openai api fine_tunes.follow -i fine_tune_id

In [ ]:
# 下面介绍一些常用的命令：
# 1）列出所有微调模型
!openai api fine_tunes.list
# 2）取消微调作业
!openai api fine_tunes.cancel -i fine_tune_id
# 3）删除微调作业
!openai api fine_tunes.delete -i fine_tune_id

# 使用微调后的模型

In [ ]:
!openai.completion.create(model="davinci:ft-book:direct-marketing-2024-04-17-12-12-12", prompt="Hotel, New York, small ->", max_tokens=100, temperature=0, stop="\n")